# CMAC 

## Import Library

In [31]:
# Imports
from Crypto.Cipher import AES
from Crypto.Util.strxor import strxor
from typing import List, Tuple, Union, Optional
import pydantic
import binascii

# For converting between different representations
def bytes_to_int_list(data: bytes) -> List[int]:
    """Convert bytes to a list of integers (32-bit chunks)"""
    result = []
    for i in range(0, len(data), 4):
        chunk = data[i:i+4].ljust(4, b'\x00')  # Ensure 4 bytes (pad if needed)
        result.append(int.from_bytes(chunk, byteorder='big'))
    return result

def int_list_to_bytes(int_list: List[int]) -> bytes:
    """Convert a list of integers (32-bit chunks) to bytes"""
    result = b''
    for value in int_list:
        result += value.to_bytes(4, byteorder='big')
    return result

def hex_to_bytes(hex_str: str) -> bytes:
    """Convert hex string to bytes"""
    return binascii.unhexlify(hex_str.replace(' ', ''))

## Pydantic Data Models
- Define models for the key and message blocks
- Support conversion between different representations (bytes, hex, integers)


In [32]:
class CmacBlock(pydantic.BaseModel):
    """A 128-bit block representation with conversion utilities"""
    # Four 32-bit words (128 bits total)
    words: Tuple[int, int, int, int]
    
    @classmethod
    def from_bytes(cls, data: bytes) -> 'CmacBlock':
        """Create a block from bytes"""
        if len(data) != 16:
            raise ValueError(f"Block must be exactly 16 bytes (got {len(data)})")
        w1 = int.from_bytes(data[0:4], byteorder='big')
        w2 = int.from_bytes(data[4:8], byteorder='big')
        w3 = int.from_bytes(data[8:12], byteorder='big')
        w4 = int.from_bytes(data[12:16], byteorder='big')
        return cls(words=(w1, w2, w3, w4))
    
    @classmethod
    def from_hex(cls, hex_str: str) -> 'CmacBlock':
        """Create a block from hex string"""
        return cls.from_bytes(hex_to_bytes(hex_str))
    
    def to_bytes(self) -> bytes:
        """Convert block to bytes"""
        result = b''
        for word in self.words:
            result += word.to_bytes(4, byteorder='big')
        return result
    
    def to_hex(self) -> str:
        """Convert block to hex string"""
        return binascii.hexlify(self.to_bytes()).decode('ascii')
    
    def print_block(self) -> None:
        """Print block in hex format for debugging"""
        print(f"0x{self.words[0]:08x} 0x{self.words[1]:08x} 0x{self.words[2]:08x} 0x{self.words[3]:08x}")


class CmacKey(pydantic.BaseModel):
    """AES key representation with support for both 128 and 256 bit keys"""
    key_bytes: bytes
    
    @property
    def size(self) -> int:
        """Get key size in bits"""
        return len(self.key_bytes) * 8
    
    @classmethod
    def from_hex(cls, hex_str: str) -> 'CmacKey':
        """Create key from hex string"""
        return cls(key_bytes=hex_to_bytes(hex_str))
    
    @classmethod
    def from_blocks(cls, blocks: List[CmacBlock]) -> 'CmacKey':
        """Create key from CmacBlocks"""
        key_bytes = b''
        for block in blocks:
            key_bytes += block.to_bytes()
        return cls(key_bytes=key_bytes)
    
    def validate_key_size(self) -> None:
        """Validate that key size is appropriate for AES"""
        valid_sizes = [128, 192, 256]  # bits
        if self.size not in valid_sizes:
            raise ValueError(f"Key size must be one of {valid_sizes} bits (got {self.size})")

## CMAC Class 
- CMAC uses AES in a special way to create a MAC
- The algorithm includes subkey generation and special handling of the final block
- The core logic maintains state between block operations

In [33]:
class CMAC:
    """CMAC implementation using AES as the underlying block cipher"""
    
    R128 = CmacBlock(words=(0, 0, 0, 0x00000087))
    AES_BLOCK_LENGTH = 128  # bits
    
    def __init__(self, verbose: bool = False):
        """Initialize CMAC instance
        
        Args:
            verbose: Whether to print debug information
        """
        self.verbose = verbose
    
    def xor_blocks(self, a: CmacBlock, b: CmacBlock) -> CmacBlock:
        """XOR two 128-bit blocks together
        
        Args:
            a: First block
            b: Second block
            
        Returns:
            Result of a XOR b
        """
        result = CmacBlock(words=(
            a.words[0] ^ b.words[0],
            a.words[1] ^ b.words[1],
            a.words[2] ^ b.words[2],
            a.words[3] ^ b.words[3]
        ))
        
        if self.verbose:
            print("XORing blocks:")
            a.print_block()
            b.print_block()
            print("Result:")
            result.print_block()
            print()
        
        return result
    
    def shift_block_left(self, block: CmacBlock) -> CmacBlock:
        """Shift a 128-bit block left by one bit
        
        Args:
            block: Block to shift
            
        Returns:
            Shifted block
        """
        # Convert block to a single large integer
        w = ((block.words[0] << 96) + 
             (block.words[1] << 64) + 
             (block.words[2] << 32) + 
              block.words[3]) & ((2**128) - 1)
        
        # Shift left by 1
        ws = (w << 1) & ((2**128) - 1)
        
        # Convert back to 4-word tuple
        return CmacBlock(words=(
            (ws >> 96) & 0xffffffff,
            (ws >> 64) & 0xffffffff,
            (ws >> 32) & 0xffffffff,
            ws & 0xffffffff
        ))
        

## Padding and Subkey Generation 
- CMAC needs special handling for partial blocks
- Two subkeys (K1, K2) are derived from the encryption key
- Padding is applied when messages don't align to block boundaries

In [34]:
class CMAC(CMAC):  # Lanjutin class definition - lanjutan dari yang di atas
    
    def pad_block(self, block: CmacBlock, bitlen: int) -> CmacBlock:
        """Pad a block with '10...0' padding based on bitlen
        
        Args:
            block: Block to pad
            bitlen: Number of valid bits in the block (0-127)
            
        Returns:
            Padded block
        """
        if not 0 <= bitlen <= 127:
            raise ValueError(f"Bit length must be between 0 and 127 (got {bitlen})")
        
        # Convert block to a single large integer
        bw = ((block.words[0] << 96) + 
              (block.words[1] << 64) + 
              (block.words[2] << 32) + 
               block.words[3]) & ((2**128) - 1)
        
        # Create a bitmask with 1s for valid bits, 0s elsewhere
        bitstr = "1" * bitlen + "0" * (128 - bitlen)
        bitmask = int(bitstr, 2)
        
        # Apply mask and add the '1' padding bit
        masked = bw & bitmask
        padded = masked + (1 << (127 - bitlen))
        
        # Convert back to 4-word tuple
        return CmacBlock(words=(
            (padded >> 96) & 0xffffffff,
            (padded >> 64) & 0xffffffff,
            (padded >> 32) & 0xffffffff,
            padded & 0xffffffff
        ))
    
    def generate_subkeys(self, key: CmacKey) -> Tuple[CmacBlock, CmacBlock]:
        """Generate subkeys K1 and K2 for CMAC
        
        Args:
            key: AES key
            
        Returns:
            Tuple of (K1, K2) blocks
        """
        # Validate key size
        key.validate_key_size()
        
        # Step 1: Encrypt a zero block with the key
        cipher = AES.new(key.key_bytes, AES.MODE_ECB)
        L_bytes = cipher.encrypt(bytes(16))  # Encrypt all-zero block
        L = CmacBlock.from_bytes(L_bytes)
        
        # Step 2: Generate K1
        Pre_K1 = self.shift_block_left(L)
        MSB_L = (L.words[0] >> 31) & 0x01
        if MSB_L:
            K1 = self.xor_blocks(Pre_K1, self.R128)
        else:
            K1 = Pre_K1
            
        # Step 3: Generate K2
        Pre_K2 = self.shift_block_left(K1)
        MSB_K1 = (K1.words[0] >> 31) & 0x01
        if MSB_K1:
            K2 = self.xor_blocks(Pre_K2, self.R128)
        else:
            K2 = Pre_K2
            
        if self.verbose:
            print("Subkey Generation:")
            print("L:")
            L.print_block()
            print(f"MSB_L = {MSB_L}")
            print("K1:")
            K1.print_block()
            print(f"MSB_K1 = {MSB_K1}")
            print("K2:")
            K2.print_block()
        
        return (K1, K2)

## Main CMAC Function

- The core CMAC algorithm processes the message blocks sequentially
- Special handling for the final block based on whether it's complete or partial
- Uses the derived subkeys to finish the MAC calculation

In [42]:
class CMAC(CMAC): 
    
    def cmac(self, key: CmacKey, message: List[CmacBlock], final_length: int) -> CmacBlock:
        """Calculate CMAC for a message
        
        Args:
            key: AES key
            message: Message as list of blocks
            final_length: Number of bits in final block (0-128)
            
        Returns:
            CMAC value as a block
        """
        # Generate subkeys
        (K1, K2) = self.generate_subkeys(key)
        if self.verbose:
            print("Subkeys generated.")
        
        # Initialize state
        state = CmacBlock(words=(0, 0, 0, 0))
        blocks = len(message)
        
        # Create AES cipher
        cipher = AES.new(key.key_bytes, AES.MODE_ECB)
        
        if blocks == 0:
            # Empty message
            padded_block = self.pad_block(state, 0)
            tweak = self.xor_blocks(padded_block, K2)
            if self.verbose:
                print("Processing empty message")
                print("Tweaked block:")
                tweak.print_block()
                
            mac_bytes = cipher.encrypt(tweak.to_bytes())
            mac = CmacBlock.from_bytes(mac_bytes)
            
        else:
            # Process all blocks except the final one
            for i in range(blocks - 1):
                state = self.xor_blocks(state, message[i])
                if self.verbose:
                    print(f"State before AES block {i+1}:")
                    state.print_block()
                    
                state_bytes = cipher.encrypt(state.to_bytes())
                state = CmacBlock.from_bytes(state_bytes)
                
                if self.verbose:
                    print(f"State after AES block {i+1}:")
                    state.print_block()
            
            # Process final block (with appropriate padding and subkey)
            if final_length == self.AES_BLOCK_LENGTH:
                # Final block is complete - use K1
                tweak = self.xor_blocks(message[-1], K1)
                if self.verbose:
                    print("Complete final block with K1:")
                    tweak.print_block()
            else:
                # Final block is partial - pad and use K2
                padded_block = self.pad_block(message[-1], final_length)
                tweak = self.xor_blocks(padded_block, K2)
                if self.verbose:
                    print("Incomplete final block with K2:")
                    tweak.print_block()
            
            # XOR state with tweaked final block and encrypt
            state = self.xor_blocks(state, tweak)
            if self.verbose:
                print("State before final AES:")
                state.print_block()
                
            mac_bytes = cipher.encrypt(state.to_bytes())
            mac = CmacBlock.from_bytes(mac_bytes)
            
            if self.verbose:
                print("MAC after final AES:")
                mac.print_block()
        
        return mac

## Utility method

In [36]:
class CMAC(CMAC):  # Continue the class definition
    
    def check_mac(self, expected: CmacBlock, result: CmacBlock) -> bool:
        """Check if a MAC matches the expected value
        
        Args:
            expected: Expected MAC
            result: Calculated MAC
            
        Returns:
            True if MACs match, False otherwise
        """
        match = (expected.words[0] == result.words[0] and 
                 expected.words[1] == result.words[1] and
                 expected.words[2] == result.words[2] and 
                 expected.words[3] == result.words[3])
        
        if self.verbose:
            if match:
                print("✅ MAC verification passed - MAC matches expected value")
            else:
                print("❌ MAC verification failed - MAC does not match expected value")
                print("Expected:")
                expected.print_block()
                print("Got:")
                result.print_block()
        
        return match
    
    def verify_message(self, key: CmacKey, message: List[CmacBlock], 
                      final_length: int, received_mac: CmacBlock) -> bool:
        """Verify a message using CMAC
        
        Args:
            key: The shared key
            message: Message blocks to verify
            final_length: Bits in final block
            received_mac: The MAC to verify against
            
        Returns:
            True if verification passes, False otherwise
        """
        calculated_mac = self.cmac(key, message, final_length)
        return self.check_mac(received_mac, calculated_mac)

## Sender and Receiver Using CMAC

### Sender

In [ ]:
print("===== CMAC Communication Example =====\n")
    
# Shared information
shared_key = CmacKey.from_hex("2b7e1516 28aed2a6 abf71588 09cf4f3c")
    
# Sender part
print("SENDER: Preparing message with authentication")
print("--------------------------------------------")

message_text = "This is a secret message that needs authentication!"
message_bytes = message_text.encode('utf-8')

# Calculate how many complete blocks we have
blocks = []
block_size_bytes = 16  # 128 bits = 16 bytes

for i in range(0, len(message_bytes), block_size_bytes):
    chunk = message_bytes[i:i+block_size_bytes]
    if len(chunk) == block_size_bytes:
        blocks.append(CmacBlock.from_bytes(chunk))
    else:
        #create a partial block
        last_block_bytes = chunk.ljust(16, b'\x00')  # Pad with zeros for the block structure
        blocks.append(CmacBlock.from_bytes(last_block_bytes))
        final_bits = len(chunk) * 8  # Convert bytes to bits

if len(message_bytes) % block_size_bytes == 0:
    # Message aligns with block boundary
    mac = cmac.cmac(shared_key, blocks, 128)
else:
    # Partial final block
    mac = cmac.cmac(shared_key, blocks, final_bits)

cmac = CMAC(verbose=True)


===== CMAC Communication Example =====

SENDER: Preparing message with authentication
--------------------------------------------
XORing blocks:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d51bc
0x00000000 0x00000000 0x00000000 0x00000087
Result:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b

Subkey Generation:
L:
0x7df76b0c 0x1ab899b3 0x3e42f047 0xb91b546f
MSB_L = 0
K1:
0xfbeed618 0x35713366 0x7c85e08f 0x7236a8de
MSB_K1 = 1
K2:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b
Subkeys generated.
XORing blocks:
0x00000000 0x00000000 0x00000000 0x00000000
0x54686973 0x20697320 0x61207365 0x63726574
Result:
0x54686973 0x20697320 0x61207365 0x63726574

State before AES block 1:
0x54686973 0x20697320 0x61207365 0x63726574
State after AES block 1:
0x65bf63df 0x7687d1c1 0xb38eda29 0xd416666d
XORing blocks:
0x65bf63df 0x7687d1c1 0xb38eda29 0xd416666d
0x206d6573 0x73616765 0x20746861 0x74206e65
Result:
0x45d206ac 0x05e6b6a4 0x93fab248 0xa0360808

State before AES block 2:
0x45d206ac 0x05e6b6a4 0x93fab248 0

In [38]:
print(f"Original message: {message_text}")
print("Generated MAC:")
mac.print_block()
print(f"MAC hex: {mac.to_hex()}")
print()

Original message: This is a secret message that needs authentication!
Generated MAC:
0xbe03541e 0x3346a188 0x47f4980d 0xf5db5969
MAC hex: be03541e3346a18847f4980df5db5969



### Receiver

In [39]:
print("RECEIVER: Verifying authentic message")
print("------------------------------------")
    
received_mac = mac  # Assume MAC was transmitted correctly
verification = cmac.verify_message(shared_key, blocks, 
                                    final_bits if len(message_bytes) % block_size_bytes != 0 else 128, 
                                    received_mac)
    
print(f"Verification result: {'✅ Authentic' if verification else '❌ Not authentic'}")
print()

RECEIVER: Verifying authentic message
------------------------------------
XORing blocks:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d51bc
0x00000000 0x00000000 0x00000000 0x00000087
Result:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b

Subkey Generation:
L:
0x7df76b0c 0x1ab899b3 0x3e42f047 0xb91b546f
MSB_L = 0
K1:
0xfbeed618 0x35713366 0x7c85e08f 0x7236a8de
MSB_K1 = 1
K2:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b
Subkeys generated.
XORing blocks:
0x00000000 0x00000000 0x00000000 0x00000000
0x54686973 0x20697320 0x61207365 0x63726574
Result:
0x54686973 0x20697320 0x61207365 0x63726574

State before AES block 1:
0x54686973 0x20697320 0x61207365 0x63726574
State after AES block 1:
0x65bf63df 0x7687d1c1 0xb38eda29 0xd416666d
XORing blocks:
0x65bf63df 0x7687d1c1 0xb38eda29 0xd416666d
0x206d6573 0x73616765 0x20746861 0x74206e65
Result:
0x45d206ac 0x05e6b6a4 0x93fab248 0xa0360808

State before AES block 2:
0x45d206ac 0x05e6b6a4 0x93fab248 0xa0360808
State after AES block 2:
0x809c17d5 0x24ac9d92

### Tampered Message

In [40]:
print("RECEIVER: Verifying tampered message")
print("-----------------------------------")
    
# Tamper with the message - change a character
tampered_text = message_text.replace("secret", "PUBLIC")
tampered_bytes = tampered_text.encode('utf-8')
    
# Recalculate blocks with tampered message
tampered_blocks = []
for i in range(0, len(tampered_bytes), block_size_bytes):
    chunk = tampered_bytes[i:i+block_size_bytes]
    if len(chunk) == block_size_bytes:
        tampered_blocks.append(CmacBlock.from_bytes(chunk))
    else:
        last_block_bytes = chunk.ljust(16, b'\x00')
        tampered_blocks.append(CmacBlock.from_bytes(last_block_bytes))
        final_bits = len(chunk) * 8

# Try to verify with original MAC
print(f"Tampered message: {tampered_text}")
verification = cmac.verify_message(shared_key, tampered_blocks, 
                                    final_bits if len(tampered_bytes) % block_size_bytes != 0 else 128, 
                                    received_mac)


RECEIVER: Verifying tampered message
-----------------------------------
Tampered message: This is a PUBLIC message that needs authentication!
XORing blocks:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d51bc
0x00000000 0x00000000 0x00000000 0x00000087
Result:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b

Subkey Generation:
L:
0x7df76b0c 0x1ab899b3 0x3e42f047 0xb91b546f
MSB_L = 0
K1:
0xfbeed618 0x35713366 0x7c85e08f 0x7236a8de
MSB_K1 = 1
K2:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b
Subkeys generated.
XORing blocks:
0x00000000 0x00000000 0x00000000 0x00000000
0x54686973 0x20697320 0x61205055 0x424c4943
Result:
0x54686973 0x20697320 0x61205055 0x424c4943

State before AES block 1:
0x54686973 0x20697320 0x61205055 0x424c4943
State after AES block 1:
0x9ce706dd 0x2c2087a3 0xeac298ba 0x5da56797
XORing blocks:
0x9ce706dd 0x2c2087a3 0xeac298ba 0x5da56797
0x206d6573 0x73616765 0x20746861 0x74206e65
Result:
0xbc8a63ae 0x5f41e0c6 0xcab6f0db 0x298509f2

State before AES block 2:
0xbc8a63ae 0x5f41e0c6 

In [41]:
# Calculate correct MAC for tampered message (to show difference)
tampered_mac = cmac.cmac(shared_key, tampered_blocks, 
                        final_bits if len(tampered_bytes) % block_size_bytes != 0 else 128)
    
print("\nComparing MACs:")
print("Original MAC:")
mac.print_block()
print("MAC for tampered message:")
tampered_mac.print_block()

XORing blocks:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d51bc
0x00000000 0x00000000 0x00000000 0x00000087
Result:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b

Subkey Generation:
L:
0x7df76b0c 0x1ab899b3 0x3e42f047 0xb91b546f
MSB_L = 0
K1:
0xfbeed618 0x35713366 0x7c85e08f 0x7236a8de
MSB_K1 = 1
K2:
0xf7ddac30 0x6ae266cc 0xf90bc11e 0xe46d513b
Subkeys generated.
XORing blocks:
0x00000000 0x00000000 0x00000000 0x00000000
0x54686973 0x20697320 0x61205055 0x424c4943
Result:
0x54686973 0x20697320 0x61205055 0x424c4943

State before AES block 1:
0x54686973 0x20697320 0x61205055 0x424c4943
State after AES block 1:
0x9ce706dd 0x2c2087a3 0xeac298ba 0x5da56797
XORing blocks:
0x9ce706dd 0x2c2087a3 0xeac298ba 0x5da56797
0x206d6573 0x73616765 0x20746861 0x74206e65
Result:
0xbc8a63ae 0x5f41e0c6 0xcab6f0db 0x298509f2

State before AES block 2:
0xbc8a63ae 0x5f41e0c6 0xcab6f0db 0x298509f2
State after AES block 2:
0x71cb7e7e 0x64b4a573 0xd8c33510 0x5fa08b9d
XORing blocks:
0x71cb7e7e 0x64b4a573 0xd8c33510 0x5f